In [7]:
!pip install -Uq nest_asyncio langchain openai lxml bs4

In [5]:
from langchain.document_loaders.sitemap import SitemapLoader
# fixes a bug with asyncio and jupyter
import nest_asyncio

nest_asyncio.apply()

In [13]:
sitemap_loader = SitemapLoader(
    web_path="https://python.langchain.com/sitemap.xml",
    )

sitemap_loader.requests_per_second = 2

# Optional: avoid `[SSL: CERTIFICATE_VERIFY_FAILED]` issue
# sitemap_loader.requests_kwargs = {"verify": False}

# docs = sitemap_loader.load()

Fetching pages: 100%|##########| 977/977 [01:33<00:00, 10.40it/s]


In [90]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100
)

data = sitemap_loader.load_and_split(text_splitter=text_splitter)

docs = text_splitter.split_documents(data)

Fetching pages: 100%|##########| 977/977 [01:25<00:00, 11.44it/s]


In [64]:
!pip install -Uq supabase tiktoken python-dotenv

In [66]:
from supabase.client import Client, create_client
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores.supabase import SupabaseVectorStore
import os
from dotenv import load_dotenv

load_dotenv()

supabase_url = os.environ.get("SUPABASE_URL")
supabase_key = os.environ.get("SUPABASE_SERVICE_KEY")
supabase: Client = create_client(supabase_url, supabase_key)

embeddings = OpenAIEmbeddings()

In [91]:
len(docs)

16864

In [92]:
vector_store = SupabaseVectorStore.from_documents(
    documents=docs[7000:],
    embedding=embeddings,
    client=supabase,
    table_name="documents",
    query_name="match_documents",
    chunk_size=100
)

2023-11-05 18:26:10,396:INFO - HTTP Request: POST https://qublpyarwoevdeqqicbz.supabase.co/rest/v1/documents "HTTP/1.1 201 Created"
2023-11-05 18:26:16,428:INFO - HTTP Request: POST https://qublpyarwoevdeqqicbz.supabase.co/rest/v1/documents "HTTP/1.1 201 Created"
2023-11-05 18:26:22,972:INFO - HTTP Request: POST https://qublpyarwoevdeqqicbz.supabase.co/rest/v1/documents "HTTP/1.1 201 Created"
2023-11-05 18:26:27,171:INFO - HTTP Request: POST https://qublpyarwoevdeqqicbz.supabase.co/rest/v1/documents "HTTP/1.1 201 Created"
2023-11-05 18:26:33,009:INFO - HTTP Request: POST https://qublpyarwoevdeqqicbz.supabase.co/rest/v1/documents "HTTP/1.1 201 Created"
2023-11-05 18:26:39,144:INFO - HTTP Request: POST https://qublpyarwoevdeqqicbz.supabase.co/rest/v1/documents "HTTP/1.1 201 Created"
2023-11-05 18:26:45,403:INFO - HTTP Request: POST https://qublpyarwoevdeqqicbz.supabase.co/rest/v1/documents "HTTP/1.1 201 Created"
2023-11-05 18:26:51,345:INFO - HTTP Request: POST https://qublpyarwoevdeqqic

In [93]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.retrievers import RePhraseQueryRetriever
from langchain.memory import ConversationBufferMemory
from langchain.chains import create_citation_fuzzy_match_chain


memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

DEFAULT_TEMPLATE = """You are an assistant tasked with taking a natural language \
query from a user and converting it into a query for a vectorstore. \
In this process, you strip out information that is not relevant for \
the retrieval task. Here is the user query: {question}"""

llm = ChatOpenAI(temperature=0)
retriever_from_llm = RePhraseQueryRetriever.from_llm(
    retriever=vector_store.as_retriever(), llm=llm
)
# docs = retriever_from_llm.get_relevant_documents("How do I load documents from Hacker News?")

# qa = ConversationalRetrievalChain.from_llm(OpenAI(temperature=0), vectorstore.as_retriever(), memory=memory)

# chain = create_citation_fuzzy_match_chain(llm)

qa = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever_from_llm, memory=memory)


In [1]:
qa({ "question": "How do I get a Retrieval chain to cite its sources?"})

NameError: name 'qa' is not defined